In [1]:
import numpy as np
from matplotlib import pyplot as plt
from NeuralNetwork import NeuralNetwork
from functions import *
from Layer import Layer, Input
from utils import *

In [2]:
df = get_data('/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/MONK/monks-1.train')
df_test = get_data('/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/MONK/monks-1.test')

In [3]:
#df = get_data(r"\Users\s512fj-ej021t\OneDrive\Desktop\ML\monks-1.train")
#df_test = get_data(r"\Users\s512fj-ej021t\OneDrive\Desktop\ML\monks-1.test")

In [4]:
X_train, y_train = df.drop(columns=['target','id']).to_numpy().T, df['target'].apply(lambda x: int(x)).to_numpy().T
X_test, y_test = df_test.drop(columns=['target','id']).to_numpy().T, df_test['target'].apply(lambda x: int(x)).to_numpy().T

In [5]:
X_train = onehot_encoding(X_train)

In [25]:
def cross_validation(input, target, folds, metrics, params, callbacks):

    dim_input = input.shape[1]
    input_index = np.arange(dim_input)
    np.random.shuffle(input_index)
    subset_index = []
    
    for i in range(folds-1):
        subset_index.append(input_index[i*np.round(dim_input / folds).astype(int): (i+1)*np.round(dim_input / folds).astype(int)])
    subset_index.append(input_index[(i+1)*np.round(dim_input / folds).astype(int):])

   
    history_cv = {'train_loss': [],
                  #'train_loss_var': [],
                  'val_loss': []
                  #'val_loss_var': []
                  }
    
    for m in metrics:
        history_cv[f'train_{m.__name__}'] = []
        #history_cv[f'train_{m.__name__}_var'] = []
        history_cv[f'val_{m.__name__}'] = []
        #history_cv[f'val_{m.__name__}_var'] = []

    for val_ind in subset_index:

        train_ind = list(set(input_index) - set(val_ind))

        train_input = input[:,train_ind]
        train_target = target[:,train_ind]
        val_input = input[:,val_ind]
        val_target = target[:,val_ind]

        input_layer = Input(17)
        hidden_layer = Layer(input_layer, 15, 'relu')
        output_layer = Layer(hidden_layer, 1, 'sigm')

        model = NeuralNetwork(input_layer, output_layer, loss = 'binary_crossentropy', metrics = metrics)

        history = model.train(train_input, train_target, **params,
                              **callbacks,
                                validation_data = [val_input, val_target],
                                verbose=0
                            )
        
        history_cv['train_loss'].append(history['train_loss'][-1])
        history_cv['val_loss'].append(history['val_loss'][-1])
        for m in metrics:
            history_cv[f'train_{m.__name__}'].append(history[f'train_{m.__name__}'][-1])
            history_cv[f'val_{m.__name__}'].append(history[f'val_{m.__name__}'][-1])
    
    history_cv['train_loss_mean'] = np.mean(history_cv['train_loss'])
    history_cv['train_loss_std'] = np.std(history_cv['train_loss'])
    history_cv['val_loss_mean'] = np.mean(history_cv['val_loss'])
    history_cv['val_loss_std'] = np.std(history_cv['val_loss'])
    del history_cv['train_loss']
    del history_cv['val_loss']

    for m in metrics:
        history_cv[f'train_{m.__name__}_mean'] = np.mean(history_cv[f'train_{m.__name__}'])
        history_cv[f'train_{m.__name__}_std'] = np.std(history_cv[f'train_{m.__name__}'])
        history_cv[f'val_{m.__name__}_mean'] = np.mean(history_cv[f'val_{m.__name__}'])
        history_cv[f'val_{m.__name__}_std'] = np.std(history_cv[f'val_{m.__name__}'])
        del history_cv[f'train_{m.__name__}']
        del history_cv[f'val_{m.__name__}']

    return history_cv
            

In [26]:
def grid_search(input, target, params, cv_folds, metrics, callbacks):
    param_grid = makeGrid(params)
    print('Starting grid_search...')
    print(f'Grid of parameters: {params}')
    print('-------------------------------------------------')
    for p_comb in param_grid:
        print(f'Starting params: {p_comb}')
        p_comb['results'] = cross_validation(input, target, cv_folds, metrics, p_comb, callbacks)
        
        print(f'Results:')
        for key, value in p_comb['results'].items():
            print(f'{key}: {value:.2e}')
        print('-------------------------------------------------')
    
    best_m = param_grid[0]['results'][f'val_accuracy_mean']
    best_comb = param_grid[0]
    for p_comb in param_grid:
        if p_comb['results'][f'val_accuracy_mean'] > best_m:
            best_m = p_comb['results'][f'val_accuracy_mean']
            best_comb = p_comb
    print(f'Best combination of parameters: {best_comb}')
    return best_comb

In [27]:
params = {'eta' : [0.01,0.02,0.03],
          'lam' : [0.0, 0.1],
          'epochs': [500],
          'n_batch' : [31]
          #alpha
          #dim hidden
        }

In [28]:
early_stopping = {'patience' : 150,
                  'monitor' : 'val_accuracy',
                  'verbose' : 0,
                  'compare_function': np.greater_equal}

reduce_eta = {'patience' : 75,
              'monitor' : 'val_accuracy',
              'factor' : 0.5,
              'verbose' : 0,
              'compare_function': np.greater_equal}

callbacks = {'early_stopping': None,
             'reduce_eta': None}

In [29]:
best_comb = grid_search(X_train, y_train.reshape((1,124)), params, 5, [accuracy], callbacks)
    

Starting grid_search...
Grid of parameters: {'eta': [0.01, 0.02, 0.03], 'lam': [0.0, 0.1], 'epochs': [500], 'n_batch': [31]}
-------------------------------------------------
Starting params: {'eta': 0.01, 'lam': 0.0, 'epochs': 500, 'n_batch': 31}
Results:
train_loss_mean: 1.04e-03
train_loss_std: 1.70e-04
val_loss_mean: 2.57e-03
val_loss_std: 5.04e-04
train_accuracy_mean: 1.00e+02
train_accuracy_std: 0.00e+00
val_accuracy_mean: 1.00e+02
val_accuracy_std: 0.00e+00
-------------------------------------------------
Starting params: {'eta': 0.01, 'lam': 0.1, 'epochs': 500, 'n_batch': 31}
Results:
train_loss_mean: 7.55e-03
train_loss_std: 5.96e-04
val_loss_mean: 1.51e-02
val_loss_std: 6.24e-03
train_accuracy_mean: 9.90e+01
train_accuracy_std: 1.25e+00
val_accuracy_mean: 9.28e+01
val_accuracy_std: 8.91e+00
-------------------------------------------------
Starting params: {'eta': 0.02, 'lam': 0.0, 'epochs': 500, 'n_batch': 31}
Results:
train_loss_mean: 6.03e-04
train_loss_std: 1.00e-04
val_

c:\Users\HP\Desktop\UNI\LM_1\MachineLearning\ML_prj\notebooks\functions.py:91: RuntimeWarning: divide by zero encountered in log
  return - (target * np.log(layer) + (1 - target) * np.log(1 - layer))/target.shape[1]
c:\Users\HP\Desktop\UNI\LM_1\MachineLearning\ML_prj\notebooks\functions.py:91: RuntimeWarning: invalid value encountered in multiply
  return - (target * np.log(layer) + (1 - target) * np.log(1 - layer))/target.shape[1]
c:\Users\HP\Desktop\UNI\LM_1\MachineLearning\ML_prj\notebooks\functions.py:94: RuntimeWarning: invalid value encountered in divide
  return (layer - target)/(layer*(1-layer)*target.shape[1])
c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: invalid value encountered in relu (vectorized)
  outputs = ufunc(*inputs)
c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: invalid value encountered in d_relu (vectorized)
  outputs = 

Results:
train_loss_mean: nan
train_loss_std: nan
val_loss_mean: nan
val_loss_std: nan
train_accuracy_mean: 8.99e+01
train_accuracy_std: 2.02e+01
val_accuracy_mean: 9.04e+01
val_accuracy_std: 1.92e+01
-------------------------------------------------
Starting params: {'eta': 0.03, 'lam': 0.1, 'epochs': 500, 'n_batch': 31}
Results:
train_loss_mean: 1.98e-03
train_loss_std: 6.09e-04
val_loss_mean: 4.22e-03
val_loss_std: 2.16e-03
train_accuracy_mean: 1.00e+02
train_accuracy_std: 0.00e+00
val_accuracy_mean: 9.83e+01
val_accuracy_std: 3.33e+00
-------------------------------------------------
Best combination of parameters: {'eta': 0.01, 'lam': 0.0, 'epochs': 500, 'n_batch': 31, 'results': {'train_loss_mean': 0.0010398758323497928, 'train_loss_std': 0.000170173258345776, 'val_loss_mean': 0.0025654108589527403, 'val_loss_std': 0.0005040053716545955, 'train_accuracy_mean': 100.0, 'train_accuracy_std': 0.0, 'val_accuracy_mean': 100.0, 'val_accuracy_std': 0.0}}


In [18]:
a = {'aa':1, 'cc':}


SyntaxError: invalid syntax (4215045574.py, line 1)

In [24]:
np.less_equal(3,2)

False